# Migration experience premium (Chapter 2) - DATA PREPARATION

In [1]:
import numpy as np
import pandas as pd
import os
os.chdir('D:\KGZ_LiK\Chapter_2')
os.getcwd()

'D:\\KGZ_LiK\\Chapter_2'

In [2]:
### Connect to Stata
import sys
sys.path.append('D:/Moved_from_C/Stata17/utilities')
from pystata import config
config.init('mp');


  ___  ____  ____  ____  ____ ©
 /__    /   ____/   /   ____/      17.0
___/   /   /___/   /   /___/       MP—Parallel Edition

 Statistics and Data Science       Copyright 1985-2021 StataCorp LLC
                                   StataCorp
                                   4905 Lakeway Drive
                                   College Station, Texas 77845 USA
                                   800-STATA-PC        https://www.stata.com
                                   979-696-4600        stata@stata.com

Stata license: Single-user 8-core , expiring  1 Jan 2025
Serial number: 501709301094
  Licensed to: 微软用户
               微软中国

Notes:
      1. Unicode is supported; see help unicode_advice.
      2. More than 2 billion observations are allowed; see help obs_advice.
      3. Maximum number of variables is set to 5,000; see help set_maxvar.


# 1. Subset from main data set - KGZ LiK 2010-2019
### Male 16-65 years old living and working in KGZ. 

In [3]:
## From main dataset we extract the respective observations
data = pd.read_stata('\KGZ_LiK\kgzlik1019.dta').set_index('index')

data = data[(data.age>=16) & (data.age<=65)]
data = data[data.np != 1]
data = data[data.mn != 1]
data = data[data.male == 1]
data.shape

D:\Moved_from_C\TEMP\ipykernel_3956\415260878.py:2: UnicodeWarning: 
One or more strings in the dta file could not be decoded using utf-8, and
so the fallback encoding of latin-1 is being used.  This can happen when a file
has been incorrectly encoded by Stata or some other software. You should verify
the string values returned are correct.
  data = pd.read_stata('\KGZ_LiK\kgzlik1019.dta').set_index('index')


(19735, 465)

# 2. Wage imputation
### About 40% of observations have missing wage values. It needs to be imputed...
### 2.1 Getting the subset of data to be imputed and saving it separately.

In [4]:
## Turning data into python dataframe
df = data.copy()
to_be_imputed = df[(df.hours>=5) & (df.employed==1)][['age', 'age2', 'exp', 'exp2', 'educc', 'dependents',
                    'urban', 'employee', 'sector', 'occupation', 'year', 
                    'region', 'hours', 'ldwage', 'ldwagew']]

# replace categorical variables to numbers
dummies = pd.get_dummies(to_be_imputed[['educc', 'sector', 'occupation']])
dt = pd.concat([to_be_imputed, dummies], axis = 'columns')
dt = dt.drop(columns = ['educc', 'sector', 'occupation', 
                   'educc_General or less (9)', 'sector_extra-territorial organizations ', 'occupation_armed forcecs ' ], 
                    axis = 'columns')
dt.shape

(11435, 40)

### 2.2 Conducting imputation of K nearest neighbor (K=3)

In [5]:
## Imputing package
from sklearn.impute import KNNImputer

In [8]:
### Implmenting imputation
imputer = KNNImputer(n_neighbors = 3, weights = 'uniform')
imputed = pd.DataFrame(imputer.fit_transform(dt), 
                        columns = dt.columns).rename(columns = {'ldwage':'imputed_ldwage', 'ldwagew': 'imputed_ldwagew'})
imputed.shape

(11435, 40)

In [16]:
imputed = pd.concat([dt.reset_index()[['index']], imputed[['imputed_ldwage', 'imputed_ldwagew']]], axis = 'columns')
imputed.shape

(11435, 3)

### 2.3 Merging imputed data to master data

In [17]:
### Merging imputed data to master
data = pd.concat([data, imputed.set_index('index')], axis = 'columns')
data.shape

(19735, 467)

In [21]:
### Saving to file
data.to_stata('data_2.dta')

D:\Moved_from_C\TEMP\ipykernel_3956\3161434775.py:2: ValueLabelTypeMismatch: 
Stata value labels (pandas categories) must be strings. Column sector_m contains
non-string labels which will be converted to strings.  Please check that the
Stata data file created has not lost information due to duplicate labels.

  data.to_stata('data_2.dta')
D:\Moved_from_C\TEMP\ipykernel_3956\3161434775.py:2: ValueLabelTypeMismatch: 
Stata value labels (pandas categories) must be strings. Column sector contains
non-string labels which will be converted to strings.  Please check that the
Stata data file created has not lost information due to duplicate labels.

  data.to_stata('data_2.dta')
D:\Moved_from_C\TEMP\ipykernel_3956\3161434775.py:2: ValueLabelTypeMismatch: 
Stata value labels (pandas categories) must be strings. Column agegrpc contains
non-string labels which will be converted to strings.  Please check that the
Stata data file created has not lost information due to duplicate labels.

  data.to_

# 3. Merging the instrument
### Instrument: Oil rent revenue per capita at 19years old

### 3.1 Getting data from World Bank as per capita oil revenue

In [25]:
### Getting data from World Bank 
import wbgapi as wb

In [26]:
### Accumulating data
colnames = ['year', 'Real GDP per Cap', 'Oil Rents %GDP']
time = range(1990,2023)
query = ['NY.GDP.PCAP.PP.CD', 'NY.GDP.PETR.RT.ZS']
df = wb.data.DataFrame(query, 'RUS', time, index = 'time').reset_index()
df.columns = colnames
df['year'] = df['year'].str.replace('YR', '').astype(int)
df['Oil Rent per Cap'] = df['Real GDP per Cap']*df['Oil Rents %GDP']/100

### Saving to file 
df[['year', 'Oil Rent per Cap']].to_csv('wdb_rus_oil_data.csv', index = False)

### 3.2 Merging oil data with master data

In [27]:
%%stata
cd "D:\KGZ_LiK\Chapter_2"

foreach num of numlist 19/21 {
	import delimited wdb_rus_oil_data.csv, clear 
	rename * =`num'
	merge 1:m year`num' using "D:\KGZ_LiK\Chapter_2\data_2.dta"
	drop _merge
	save data_2.dta, replace
}


. cd "D:\KGZ_LiK\Chapter_2"
D:\KGZ_LiK\Chapter_2

. 
. foreach num of numlist 19/21 {
  2.         import delimited wdb_rus_oil_data.csv, clear 
  3.         rename * =`num'
  4.         merge 1:m year`num' using "D:\KGZ_LiK\Chapter_2\data_2.dta"
  5.         drop _merge
  6.         save data_2.dta, replace
  7. }
(encoding automatically selected: ISO-8859-1)
(2 vars, 33 obs)
(variable year19 was int, now float to accommodate using data's values)

    Result                      Number of obs
    -----------------------------------------
    Not matched                         6,443
        from master                         1  (_merge==1)
        from using                      6,442  (_merge==2)

    Matched                            13,293  (_merge==3)
    -----------------------------------------
file data_2.dta saved
(encoding automatically selected: ISO-8859-1)
(2 vars, 33 obs)
(variable year20 was int, now float to accommodate using data's values)

    Result                

### 3.3 Check correlation of instrument with dummy of return migration

In [28]:
%%stata
## Check correlations with instrument
global x exp exp2 i.educc urban i.region i.year hours
global ins oilrentpercap19 oilrentpercap20 oilrentpercap21

eststo clear
foreach iv in $ins {
	qui reg mp `iv' $x if imputed_ldwage!=.
	eststo 
}
esttab, b se(4) ar2 keep($ins)


. ## Check correlations with instrument
Unknown #command
. global x exp exp2 i.educc urban i.region i.year hours

. global ins oilrentpercap19 oilrentpercap20 oilrentpercap21

. 
. eststo clear

. foreach iv in $ins {
  2.         qui reg mp `iv' $x if imputed_ldwage!=.
  3.         eststo 
  4. }
(est1 stored)
(est2 stored)
(est3 stored)

. esttab, b se(4) ar2 keep($ins)

------------------------------------------------------------
                      (1)             (2)             (3)   
                       mp              mp              mp   
------------------------------------------------------------
oilrentpe~19    0.0000792***                                
                 (0.0000)                                   

oilrentpe~20                    0.0000624***                
                                 (0.0000)                   

oilrentpe~21                                    0.0000823***
                                                 (0.0000)   
-----------

# Data is ready to be used!